In [3]:
# 必要最低限
import numpy as np
import mnist
import matplotlib.pyplot as plt
from pylab import cm

X = mnist.download_and_parse_mnist_file("train-images-idx3-ubyte.gz")
Y = mnist.download_and_parse_mnist_file("train-labels-idx1-ubyte.gz") 

#入力層のクラス
class InputLayer:
    def __init__(self, x):
        self.x = x
    def output(self):
        return self.x
    
# 畳み込み層のクラス
class ConvolutionLayer:
    def __init__(self, x, filter_mateix, bias, batch, R, s):
        self.x = x #batch * ch * dx * dy行列
        self.filter_matrix = filter_mateix #フィルター行列だが、R*Rではない。フィルター行列の集合
        self.bias = bias #バイアス
        self.batch = batch #バッチサイズ
        self.R = R #フィルターサイズ
        self.s = s #ストライド
    def padding(self):
        batch, ch, dx, dy = self.x.shape
        r = int(self.R/2)
        padding = np.zeros((batch, ch, dx + 2*r, dy + 2*r)) 
        padding[:, :, r:(dx+r), r:(dy+r)] = self.x
        return padding
    def rerange(self):
        batch, ch, dx, dy = self.x.shape
        C = self.padding()
        C = np.lib.stride_tricks.as_strided(C, shape=(batch, ch, int(dx/self.s), int(dy/self.s), self.R, self.R),\
                                            strides=(C.strides[0], C.strides[1],self.s*C.strides[2], self.s*C.strides[3], C.strides[2], C.strides[3])
                                           )
        C = C.transpose(1, 4, 5, 0, 2, 3).reshape(ch * self.R * self.R, -1)
        return C
    def output(self):
        W = self.filter_matrix
        B = self.bias
        return np.dot(W, self.rerange()) + B[:,np.newaxis]

# Convolutionレイヤの実装
class Convolution:
    
    # インスタンス変数の定義
    def __init__(self, W, b, stride=1, pad=1):
        self.W = W # フィルター(重み)
        self.b = b # バイアス
        self.stride = stride # ストライド
        self.pad = pad # パディング
        
        # (逆伝播時に使用する)中間データを初期化
        self.x = None # 入力データ
        self.col = None # 2次元配列に展開した入力データ
        self.col_W = None # 2次元配列に展開したフィルター(重み)
        
        # 勾配に関する変数を初期化
        self.dW = None # フィルター(重み)に関する勾配
        self.db = None # バイアスに関する勾配
    
    # 順伝播メソッドの定義
    def forward(self, x):
        # 各データに関するサイズを取得
        FN, C, FH, FW = self.W.shape # フィルター
        N, C, H, W = x.shape # 入力データ
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride) # 出力データ:式(7.1)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)
        
        # 各データを2次元配列に展開
        col = im2col(x, FH, FW, self.stride, self.pad) # 入力データ
        col_W = self.W.reshape(FN, -1).T # フィルター
        
        # 出力の計算:(図7-12)
        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        # (逆伝播時に使用する)中間データを保存
        self.x = x
        self.col = col
        self.col_W = col_W
        
        return out
    
    # 逆伝播メソッドの定義
    def backward(self, dout):
        # フィルターに関するサイズを取得
        FN, C, FH, FW = self.W.shape
        
        # 順伝播の入力を展開
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)
        
        # 各パラメータの勾配を計算:式(5.13)
        self.db = np.sum(dout, axis=0) # バイアス
        self.dW = np.dot(self.col.T, dout) # (展開した)重み
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW) # 本来の形状に変換
        dcol = np.dot(dout, self.col_W.T) # (展開した)入力データ
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad) # 本来の形状に変換

        return dx
    
# プーリング層のクラス
class PoolingLayer:
    def __init__(self, x, batch, R, s):
        self.x = x #畳み込み層からの入力
        self.batch = batch #バッチサイズ
        self.R = R #プーリング層のサイズ
        self.s = s #ストライド
        self.max_indices = None  # 最大値のインデックスを保持する変数
    def output(self):
        dx, dy = self.x.shape
        newdy = int(np.sqrt(int(dy/self.batch)))
        #これにより batch * K * dx * dy配列に変化
        tensor_x = np.array(np.array_split(self.x, dy // int(dy/self.batch), axis=1)).reshape(self.batch, dx, newdy, newdy)
        #これにpoolingを適用する
        d = self.R
        n, c, h, w = tensor_x.shape
        output_h = (h - d) // self.s + 1 #出力の高さ
        output_w = (w - d) // self.s + 1 #出力の幅
        pooled_data = tensor_x[:, :, :output_h * self.s, :output_w * self.s].reshape(n, c, output_h, d, output_w, d)
        pooled_data = pooled_data.max(axis=(3, 5))
        return pooled_data
    def max_index(self):
        dx, dy = self.x.shape
        newdy = int(np.sqrt(int(dy/self.batch)))
        #これにより batch * K * dx * dy配列に変化
        tensor_x = np.array(np.array_split(self.x, dy // int(dy/self.batch), axis=1)).reshape(self.batch, dx, newdy, newdy)
        #これにpoolingを適用する
        d = self.R
        n, c, h, w = tensor_x.shape
        output_h = (h - d) // self.s + 1 #出力の高さ
        output_w = (w - d) // self.s + 1 #出力の幅
        pooled_data = tensor_x[:, :, :output_h * self.s, :output_w * self.s].reshape(n, c, output_h, d, output_w, d)
        # マックスプーリングのマスクを作成、多分これで、インデックスを保持することができているはず、、、
        mask = (pooled_data == np.max(pooled_data, axis=(3, 5), keepdims=True))
        # mask = np.argwhere(pooled_data == np.max(pooled_data, axis=(3, 5)))
        mask = mask.reshape(n, c, output_h*d, output_w*d).astype(int)
        return mask
    


# 全結合層のクラス
class ConnectionLayer:
    def __init__(self, x, w, b):
        self.x = x
        self.w= w
        self.b = b
    def output(self):
        self.x = np.array(self.x)
        self.w = np.array(self.w)
        self.b = np.array(self.b)
        return np.dot(self.w.T, self.x) + self.b[:, np.newaxis]
    def dot_wx(self):
        self.x = np.array(self.x)
        self.w = np.array(self.w)
        return np.dot(self.w.T, self.x)
    def dot_b(self):
        self.b = np.array(self.b)
        return self.b[:, np.newaxis]
    def test_output(self):
        self.x = np.array(self.x)
        self.w = np.array(self.w)
        self.b = np.array(self.b)
        return np.dot(self.w.T, self.x) + self.b

#中間層のクラス(sigmoid, relu)
class MiddleLayer:
    def __init__(self, x, function):
        self.x = x
        self.function = function
    def output(self):
        return self.function(self.x)
    
# 中間層のクラス（dropout）
class Dropout:
    def __init__(self, x, rho):
        self.x = x
        self.rho = rho
    def output(self):
        rows, cols = self.x.shape
        indices = np.arange(rows)
        np.random.shuffle(indices)
        result_matrix = np.copy(self.x)
        for col in range(cols):
            selected_indices = indices[:self.rho]
            result_matrix[selected_indices, col] = 0
        return result_matrix
    
# 出力層のクラス
class OutputLayer:
    def __init__(self, x, softmax):
        self.x = x
        self.softmax = softmax
    def output(self):
        return self.softmax(self.x)

# クロスエントロピーを計算するクラス
class CrossEntropy:
    def __init__(self, x, onehot):
        self.x = x
        self.onehot = onehot
    def safelog(self):
        try:
            result = -np.log(self.x)
        except RuntimeWarning as e:
            result = np.where(self.x <= 0, 100, -np.log(self.x))
        return result
    def result(self):
        loglist = self.safelog()
        crossentropy = np.sum(self.onehot * loglist, axis=0)
        batch = loglist.shape[1]
        return np.sum(crossentropy) / batch

# softmaxとクロスエントロピー誤差の逆伝播クラス
class ErrorBackCrossEntropy:
    def __init__(self, x, onehot):
        self.x = x
        self.onehot = onehot
    def del_en_x(self):
        batch = self.x.shape[1]
        return (self.x - self.onehot) / batch
    
# 中間層の逆伝播のクラス
# シグモイド関数の逆伝播のクラス
class ErrorBackSigmoid:
    def __init__(self, y, del_en_y):
        self.y = y
        self.del_en_y = del_en_y
    def del_en_x(self):
        new_y = (1 - self.y) * self.y
        result = self.del_en_y * new_y
        return result
    
# relu関数の逆伝播のクラス
class ErrorBackReLU:
    def __init__(self, y, del_en_y):
        self.y = y
        self.del_en_y = del_en_y
    def del_en_x(self):
        new_y = np.where(self.y > 0, 1, 0)
        result = self.del_en_y * new_y
        return result
    
# dropoutの逆伝播クラス
class ErrorBackDropout:
    def __init__(self, y, del_en_y):
        self.y = y
        self.del_en_y = del_en_y
    def del_en_x(self):
        new_y = np.where(self.y == 0, 0, 1)
        result = self.del_en_y * new_y
        return result
    
#正規化層の誤差逆伝播クラス
class ErrorBackNormalize:
    def __init__(self, x, x_hat, del_en_y, gamma, beta, epsilon, mean, var, batch):
        self.x = x
        self.x_hat = x_hat
        self.del_en_y = del_en_y
        self.gamma = gamma
        self.beta = beta
        self.epsilon = epsilon
        self.mean = mean
        self.var = var
        self.batch = batch
    def del_en_x_hat(self):
        return self.del_en_y * self.gamma[:,np.newaxis]
    def del_en_var(self):
        mult = np.sum(self.del_en_x_hat()*(self.x - self.mean[:, np.newaxis]), axis=1)
        return (-1) / 2 * mult *  np.power((self.var + self.epsilon), -1.5 ) 
    def del_en_mean(self):
        e1 = np.sum(self.del_en_x_hat(), axis=1)  * (-1) / np.sqrt(self.var + self.epsilon)
        e2 = (-2) * self.del_en_var() * (np.sum(self.x, axis=1)/self.batch - self.mean)
        return e1 + e2
    def del_en_x(self):
        e1 = self.del_en_x_hat() / np.sqrt(self.var + self.epsilon)[:,np.newaxis]
        e2 = 2 / self.batch * (self.x - self.mean[:,np.newaxis]) * self.del_en_var()[:,np.newaxis]
        e3 = self.del_en_mean() / self.batch
        return e1 + e2 + e3[:,np.newaxis]
    def del_en_gamma(self):
        return np.sum(self.del_en_y * self.x, axis=1)
    def del_en_beta(self):
        return np.sum(self.del_en_y, axis=1)
    
# 全結合層の逆伝播クラス
class ErrorBackConnect:
    def __init__(self, x, del_en_y, w):
        self.x = x
        self.del_en_y = del_en_y
        self.w = w
    def del_en_x(self):
        return np.dot(self.w, self.del_en_y)
    def del_en_w(self):
        return np.dot(self.del_en_y, self.x.T)
    def del_en_b(self):
        return np.sum(self.del_en_y, axis=1)
    
# プーリング層の誤差逆伝播クラス
class ErrorBackPooling:
    def __init__(self, x, y, del_en_y, K, batch, max_index, R, R_conv):
        self.x = x #プーリング層の入力
        self.y = y #プーリング層の出力
        self.del_en_y = del_en_y
        self.K = K #フィルタ数
        self.batch = batch
        self.max_index = max_index #プーリング層で抽出された要素は1, そうでない要素は0となっている配列
        self.R = R #プーリングのサイズ
        self.R_conv = R_conv
    def reshape_del(self, m):
        self.m = m
        z, batch = self.m.shape
        dx = int(np.sqrt(int(z/self.K)))
        #これにより、batch*K*dx*dx配列に変化
        tensor_del = self.m.T.reshape(batch, self.K, dx, dx)
        return tensor_del
    def reshape_x(self, n):
        self.n = n
        dx, dy = self.n.shape
        newdy = int(np.sqrt(int(dy/self.batch)))
        #これにより batch * K * dx * dy配列に変化
        tensor_x = np.array(np.array_split(self.n, dy // int(dy/self.batch), axis=1)).reshape(self.batch, dx, newdy, newdy)
        #これにpoolingを適用する
        return tensor_x
    #こちらは、pooling層2の誤差逆伝播に限定することにする
    def del_en_x(self):
        del_en_y_reshaped = self.reshape_del(self.del_en_y)
        b, k, dx, dy =  self.reshape_x(self.x).shape
        expanded_del = del_en_y_reshaped.repeat(self.R, axis=2).repeat(self.R, axis=3)
        expanded_del = expanded_del.reshape(b, k, dx, dy)
        C = expanded_del * self.max_index
        return C.transpose(1, 0, 2, 3).reshape(k, -1)
    #こちらはpooling層1の誤差逆伝播に限定
    def del_en_x_sub(self):
        # b, k, dx, dy = self.y.shape
        b, k, h, w = self.reshape_x(self.x).shape
        # expanded_del = self.del_en_y[np.arange(self.del_en_y.shape[0]) % 9 == 4]
        # expanded_del = self.reshape_del(expanded_del)
        # expanded_del = expanded_del.repeat(self.R, axis=2).repeat(self.R, axis=3)
        # expanded_del = expanded_del.reshape(b, k, w, h)
        # dx, dy = self.x.shape
        expanded_del = col2im(self.del_en_y, self.y.shape, 3, 3, 1, 1)
        expanded_del = expanded_del.repeat(self.R, axis=2).repeat(self.R, axis=3)
        expanded_del = expanded_del.reshape(b, k, w, h)
        
        C = expanded_del * self.max_index
        return C.transpose(1, 0, 2, 3).reshape(k, -1)
    
# 畳み込み層の誤差逆伝播クラス
class ErrorBackConvolution:
    def __init__(self, x, del_en_y, w):
        self.x = x
        self.del_en_y = del_en_y
        self.w = w
    def del_en_x(self):
        return np.dot(self.w.T, self.del_en_y)
    def del_en_w(self):
        return np.dot(self.del_en_y, self.x.T)
    def del_en_b(self):
        return np.sum(self.del_en_y, axis=1)
    def del_en_x_sub(self):
        del_y = np.dot(self.w.T, self.del_en_y)
        expanded_del = del_y[np.arange(del_y.shape[0]) % 9 == 4]
        return expanded_del
    
# シグモイド関数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# ソフトマックス関数
def softmax(x):
    x = x - np.max(x, axis = 0)
    return np.exp(np.array(x)) / np.sum(np.exp(np.array(x)), axis=0)

# リストの一致率を示す関数
def calculate_similarity(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("リストの長さが異なります")

    # 一致した要素の数を数える
    match_count = sum(1 for a, b in zip(list1, list2) if a == b)

    # 一致率を計算
    similarity = (match_count / len(list1)) * 100  # パーセントで表示

    return similarity

# ReLU関数
def relu(x):
    return np.where(x > 0, x, 0)


In [4]:
import numpy as np
#ファイルの読み込み
with open("le4MNIST_X (1).txt", "r") as file:
    file_contents = file.read().splitlines()

data_list = []
data_list2 = []
for item in file_contents:
    numbers = [int(num) for num in item.split()]
    reshaped_data = np.array(numbers, dtype=int).reshape(1, 28, 28)
    data_list.append(reshaped_data)
    data_list2.append(np.array(numbers))
final_data = np.array(data_list)
final_data2 = np.array(data_list2).T
print(final_data2.shape)

(784, 10000)


In [5]:
#Mnistコンテスト、畳み込みなし
# 学習モデルは3NN、バッチ正規化を全結合層1と中間層の間で行う。
# 中間層の活性化関数にはシグモイド関数を用いる
def mnist_contest_noconv(test_data, w1, b1, w2, b2, gamma, beta, mean, var):
    # イプシロンの値
    epsilon = 0.001
    # batch, ch, dx, dy = test_data.shape
    #正規化
    test_data = test_data/255
    #入力層
    input_layer = InputLayer(test_data)
    #全結合層1
    connection1_layer = ConnectionLayer(input_layer.output(), w1, b1)
    #正規化
    e1 = (gamma / np.sqrt(var + epsilon))[:,np.newaxis] * connection1_layer.output()
    e2 = beta - gamma * mean / np.sqrt(var + epsilon)
    normalized_output = e1 + e2[:,np.newaxis]
    #中間層
    middle_layer = MiddleLayer(normalized_output, sigmoid)
    #全結合層2
    connection2_layer = ConnectionLayer(middle_layer.output(), w2, b2)
    #出力層
    output_layer = OutputLayer(connection2_layer.output(), softmax)
    print(np.argmax(output_layer.output(), axis=0).tolist())
    with open("predict.txt", "w") as file:
        for item in np.argmax(output_layer.output(), axis=0).tolist():
            file.write(str(item))
    
w1 = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/w1.npy')
b1 = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/b1.npy')
w2 = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/w2.npy')
b2 = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/b2.npy')
gamma = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/gamma.npy')
beta = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/beta.npy')
test_mean = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/test_mean.npy')
test_var = np.load('/Users/mst923/OneDrive - Kyoto University/3年後期/計算機科学実験4/画像処理/mnist_contest_noconv/test_var.npy')
print("テスト開始します")
mnist_contest_noconv(final_data2,w1, b1, w2, b2, gamma, beta, test_mean, test_var)

テスト開始します
[9, 5, 6, 5, 7, 5, 2, 5, 7, 7, 7, 9, 7, 3, 5, 0, 6, 1, 6, 4, 2, 9, 4, 3, 9, 0, 3, 0, 9, 7, 5, 9, 4, 8, 4, 2, 7, 8, 5, 5, 5, 5, 5, 8, 4, 3, 8, 8, 0, 8, 3, 6, 5, 0, 4, 2, 8, 7, 3, 7, 2, 6, 5, 9, 5, 0, 7, 4, 7, 2, 7, 8, 1, 3, 9, 1, 4, 2, 7, 3, 8, 7, 4, 7, 2, 4, 9, 0, 7, 2, 6, 3, 4, 7, 7, 0, 1, 3, 4, 1, 4, 5, 7, 2, 1, 9, 8, 2, 2, 3, 6, 5, 8, 8, 7, 7, 7, 2, 2, 6, 9, 3, 9, 0, 5, 2, 4, 8, 5, 2, 5, 6, 6, 6, 6, 2, 2, 3, 6, 7, 9, 8, 5, 7, 5, 2, 6, 6, 8, 5, 3, 1, 2, 5, 8, 3, 5, 4, 6, 2, 7, 5, 6, 4, 0, 4, 8, 5, 5, 6, 5, 3, 7, 9, 5, 3, 5, 0, 2, 9, 0, 4, 4, 5, 0, 5, 0, 6, 1, 2, 3, 3, 0, 3, 4, 8, 3, 7, 5, 5, 3, 6, 0, 8, 5, 7, 6, 5, 5, 3, 3, 2, 6, 3, 7, 2, 6, 6, 3, 3, 7, 2, 4, 2, 1, 7, 3, 3, 8, 5, 5, 7, 2, 3, 8, 8, 5, 7, 4, 6, 8, 7, 2, 3, 5, 3, 7, 4, 4, 7, 3, 6, 8, 2, 7, 9, 5, 0, 6, 8, 9, 0, 5, 5, 2, 3, 6, 4, 1, 2, 9, 5, 3, 9, 7, 9, 5, 7, 0, 3, 3, 4, 5, 9, 9, 3, 5, 7, 3, 3, 5, 5, 5, 0, 8, 5, 3, 2, 3, 3, 0, 6, 5, 9, 6, 4, 3, 4, 6, 3, 4, 3, 1, 3, 0, 6, 7, 2, 3, 9, 7, 2, 5, 0, 9, 9, 4, 3, 5, 4, 